In [1]:
import numpy as np


In [2]:
import tensorflow as tf

In [3]:
from sklearn import datasets
iris=datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

In [4]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]
print(len(X))

150


In [5]:
print(type(iris.data))
print(iris.data.shape)
print((iris.data.ndim))

<class 'numpy.ndarray'>
(150, 4)
2


In [6]:
X = iris["data"][:, (2, 3)]  # petal length, petal width

In [7]:
X[:5]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2]])

In [8]:
X=X.astype('float32')
print(X.dtype)
print(X.shape)

float32
(150, 2)


In [9]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [10]:
np.random.seed(2042)

In [11]:
X_with_bias[:5]

array([[1.        , 1.39999998, 0.2       ],
       [1.        , 1.39999998, 0.2       ],
       [1.        , 1.29999995, 0.2       ],
       [1.        , 1.5       , 0.2       ],
       [1.        , 1.39999998, 0.2       ]])

In [12]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

In [13]:
print(train_size)
print(test_size)

90
30


In [14]:
X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [15]:
#X_train_wo_bias = X[rnd_indices[:train_size]]

In [16]:
#X_train_wo_bias[:5]

In [15]:
X_train[:5]

array([[1.        , 1.39999998, 0.2       ],
       [1.        , 4.0999999 , 1.29999995],
       [1.        , 5.19999981, 2.        ],
       [1.        , 4.        , 1.29999995],
       [1.        , 4.0999999 , 1.29999995]])

In [16]:
X_train.shape

(90, 3)

In [17]:
np.unique(y_train)

array([0, 1, 2])

In [18]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [19]:
y_train[:10]

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 0])

In [20]:
to_one_hot(y_train[:7])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [21]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [22]:
Y_train_one_hot.shape

(90, 3)

In [23]:
Y_train_one_hot[:5]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [24]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [25]:
n_inputs = X_train.shape[1] # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))   # == 3 (3 iris classes)

In [26]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)
old_Theta=Theta
for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - Y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients
    if iteration ==0:
        print(iteration,gradients)
        print(iteration,Theta)

0 5.446205779299409
0 [[ 0.59406399 -0.28615298 -0.30791101]
 [ 3.17973972 -1.34526175 -1.83447798]
 [ 1.10140275 -0.43091503 -0.67048771]]
0 [[ 0.10736297 -0.23166202 -0.20466374]
 [ 0.40253506 -0.65301865 -0.69922576]
 [ 1.00783577  0.41676141 -0.74347951]]
500 0.8350062644113228
1000 0.6878801462436703
1500 0.6012379159284251
2000 0.5444496888309313
2500 0.5038530211914799
3000 0.4729229006455079
3500 0.4482424456589629
4000 0.4278651134705443
4500 0.410600718643495
5000 0.39567804213167


In [27]:
Thetaa = tf.get_variable( "weightt" , initializer = tf.constant(old_Theta) )

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
logitts = tf.matmul(X_train, Thetaa)
y=tf.nn.softmax(logitts)
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logitts, labels=Y_train_one_hot, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
grads=optimizer.compute_gradients(loss)
training_op = optimizer.minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [33]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss) )
    for iteration in range(n_iterations):
        gradss=sess.run(grads)
        sess.run(training_op)
        if iteration % 500 ==0:
            print(iteration, sess.run(loss))
            print('grads=',gradss[0][0])
            print('var=',gradss[0][1])
            print("after train:")
            print(Thetaa.eval())
            print(" ")

5.447902419971495
0 5.2714679707861904
grads= [[ 0.59406399 -0.28615298 -0.30791101]
 [ 3.17973972 -1.34526175 -1.83447798]
 [ 1.10140275 -0.43091503 -0.67048771]]
var= [[ 0.11330361 -0.23452355 -0.20774285]
 [ 0.43433246 -0.66647126 -0.71757054]
 [ 1.0188498   0.41245226 -0.75018439]]
after train:
[[ 0.10736297 -0.23166202 -0.20466374]
 [ 0.40253506 -0.65301865 -0.69922576]
 [ 1.00783577  0.41676141 -0.74347951]]
 
500 0.8346161590281729
grads= [[-0.13595679  0.02841431  0.10754248]
 [ 0.01794235 -0.01436509 -0.00357726]
 [ 0.05473265  0.01235083 -0.06708348]]
var= [[ 0.66543331 -0.31522188 -0.67917422]
 [-0.7176651  -0.28252865  0.0504844 ]
 [ 0.34684639  0.45492834 -0.12065707]]
after train:
[[ 0.66679287 -0.31550602 -0.68024965]
 [-0.71784452 -0.282385    0.05052017]
 [ 0.34629907  0.45480483 -0.11998623]]
 
1000 0.6876613366847472
grads= [[-0.10042224  0.02081562  0.07960662]
 [ 0.01573514 -0.01335253 -0.00238261]
 [ 0.04034822  0.01283161 -0.05317983]]
var= [[ 1.25032975 -0.43868